In [3]:
#from HMM import hmm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, roc_auc_score, mean_squared_error

np.random.seed(1000)
folder = "/Users/pedroantonio/Desktop/BKT implementations/Bayesian-Knowledge-Tracing-master/bkt/"   

#Objetivo para la ganancia de aprendizaje
matrizInicial = np.load(folder + "TutorialKnowledgeMatrix.npy") 
matrizIntermediate = np.load(folder + "intermediateKnowledgeMatrix.npy")
print(matrizInicial)
#Crea los conjuntos de train/test
kf = KFold(n_splits=5, shuffle=True)

matrizTrain_pred = []
matrizTrain_actual = []
matrizTest_pred = []
matrizTest_actual = []

#Número de knowledge components (Ahora mismo 1, a la espera de identificar más)
numkc = 1

#Se itera para cada combinación diferente de train test
for train_index, test_index in kf.split(matrizInicial):
    
    #Se divide la matriz inicial en train test
    matrizInicial_train = matrizInicial[train_index]
    matrizInicial_test = matrizInicial[test_index]
    
    matrizIntermediate_train = matrizIntermediate[train_index]
    matrizIntermediate_test = matrizIntermediate[test_index]
    
    # Simbolos: 1 incorrecto, 2 correcto
    symbols = [['1', '2']]
    
    #Objeto HMM pasando como parámetro: 
    # Probabilidad Inicial PI --> P(L0) 1-P(L0)
    # Calculada en base al número de aciertos de los puzzles de tutorial
    # Probabilidad de cambiar de un estado a otro T --> A
    # Probabilidad de Emission E --> B 
    # Símbolos anteriores
    h = hmm(2, Pi=np.array([0.48, 0.52]), T=np.array([[1, 0], [0.4, 0.6]]),E=[np.array([[0.95, 0.05], [0.05, 0.95]])], obs_symbols=symbols)
    #h = hmm(2, Pi=np.array([0.8, 0.2]), T=np.array([[1, 0], [0.4, 0.6]]), obs_symbols=symbols)

    #Inicialización a vacío
    conjunto_train = [[] for x in range(numkc)]
    conjunto_test = [[] for x in range(numkc)]
    
    
    #Iteración para cada kc, en este caso 1 ya que no se ha hecho diferenciación de puzzles 
    for i in range(numkc):
        
        #Matriz con lista de respuestas por alumno
        datosPuzzles = np.load(folder + "IntermediatePuzzlesResults.npy", allow_pickle=True)
        print(datosPuzzles)
        #Separación de matriz de alumnos en train y test
        datosPuzzles_train, datosPuzzles_test = datosPuzzles[train_index], datosPuzzles[test_index]
        
        #Se almacena el train y test para su análisis
        train = [each for each in datosPuzzles_train if each]
        test = [each for each in datosPuzzles_test if each]
        
        #Ajusta los parámetros del modelo para maximizar la probabilidad de la secuencia de observaciones
        if train and test:
            h.baum_welch(train, debug=False)   
        
        #Con el entrenamiento del modelo, se aplica a los datos del puzzle para obtener la probabilidad del siguiente caso
        conjunto_train[i].extend(h.predict_nlg(datosPuzzles_train))
        conjunto_test[i].extend(h.predict_nlg(datosPuzzles_test))

    #print(conjunto_train)
    #print("Tamaño conjunto train: ", len(conjunto_train[0]))
    #print("Conjunto test: ",conjunto_test )
    #print("Tamaño conjunto test: ", len(conjunto_test[0]))    
        
    conjunto_train = np.transpose(conjunto_train)
    conjunto_test = np.transpose(conjunto_test)

    conjunto_train = pd.DataFrame(conjunto_train).fillna(value=0)
    conjunto_test = pd.DataFrame(conjunto_test).fillna(value=0)
    
    
    # Regresión logística para aplicar el modelo en la matriz inicial
    #print("*************Comienza la regresión logística******************")
    logreg = LogisticRegression()
    #Se entrena con las probabilidades calculadas
    logreg.fit(conjunto_train, pd.DataFrame(matrizInicial_train))
    #Se predice el cambio de la matriz inicial con las probabilidades de aprendizaje
    predict = logreg.predict(conjunto_train)
    
    #matrizTrain_pred.extend([each for each in predict])
    matrizTrain_pred = predict
    print(matrizTrain_pred)

    #Se coge la parte de entrenamiento de la matriz inicial
    matrizTrain_actual = matrizInicial_train

    predict = logreg.predict(conjunto_test)
    #matrizTest_pred.extend([each for each in predict])
    matrizTest_pred = predict
    matrizTest_actual = matrizInicial_test
    print(matrizIntermediate)

print (" ")
print ("<<<<<<< Student Learning Gain >>>>>>>")


#########################################################

matrizRes = matrizTrain_pred - matrizTrain_actual
#print(matrizTrain_pred)
#print(matrizTrain_actual)
#print(matrizRes)

cont1=0
cont0=0
contM=0
sumaTotal=0


for i in matrizTrain_pred: 
    sumaTotal = i + sumaTotal
    if( i == 1):
        cont1=cont1+1
    elif(i == 0):
        cont0=cont0+1
    else: contM = contM+1
        
#print("1´s: ", cont1)
#print("0´s: ", cont0)
#print("-1: ", contM)
#print("Suma total: ", sumaTotal)
#print("Learning gain: ", sumaTotal/len(matrizTrain_pred))
#print(" ")


cont1=0
cont0Off=0
contM=0
sumaTotal=0


for i in matrizTrain_actual: 
    sumaTotal = i + sumaTotal
    if( i == 1):
        cont1=cont1+1
    elif(i == 0):
        cont0Off=cont0Off+1
    else: contM = contM+1
        
#print("1´s: ", cont1)
#print("0´s: ", cont0Off)
#print("-1: ", contM)
#print("Suma total: ", sumaTotal)
#print("Learning gain: ", sumaTotal/len(matrizRes))
#print(" ")

cont1=0
cont0=0
contM=0
sumaTotal=0


for i in matrizRes: 
    sumaTotal = i + sumaTotal
    if( i == 1):
        cont1=cont1+1
    elif(i == 0):
        cont0=cont0+1
    else: contM = contM+1
        

print("Learning gain: ", round(sumaTotal/cont0Off, 2))



matrizError = matrizTrain_pred - matrizIntermediate_train
#print(matrizError)
cont = 0
for i in matrizError:
    if(i != 0):
        cont+=1
        
#print(cont/len(matrizError))

########################################################


print ("<<<<<<< Student Modeling >>>>>>> ")
#print ("Training MSE: ", mean_squared_error(matrizTrain_pred,matrizIntermediate_train))
print ("Difference between predicted matrix and matrix of intermediate puzzles: ", mean_squared_error(matrizIntermediate_test, matrizTest_pred))
print(" ")





[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1.
 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0.]


************************
HMM Initialization
************************

1) Numerber of

-713.1116952741332
-711.9904307932774
-711.410301462711
-711.0620142041344
-710.8396611150598
-710.6934430147278
-710.5955085273041
-710.529029933687
-710.4834303650777
-710.4518745797824
-710.4298397367121
-710.414269055999
-710.4030565126413
-710.3947252234968
-710.3882221015688
-710.3827840081087
-710.3778488720532
-710.3729956765869
-710.3679033236514
-710.3623220526413
-710.3560533363105
-710.348935581661
-710.3408338576153
-710.3316324496674
-710.3212294215364
-710.3095326167066
-710.296456703397
-710.2819209829248
-710.2658477620222
-710.2481611458634
-710.2287861484574
-710.2076480455044
-710.1846719155554
-710.159782330581
-710.1329031685258
-710.1039575293096
-710.0728677427128
-710.0395554624187
-710.0039418453964
-709.9659478202527
-709.9254944522911
-709.8825034169713
-709.8368975973586
-709.7886018249746
-709.7375437872718
-709.6836551285712
-709.6268727745795
-709.5671405132209
-709.5044108660992
-709.4386472847053
-709.3698267028789
-709.2979424708111
-709.2230076848895

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-721.2109459537796
-718.3590147299545
-717.3484571560584
-716.8259015836135
-716.4903980573102
-716.2517668971653
-716.0741911715245
-715.9390088498919
-715.8344874474773
-715.7525435306383
-715.6873934517994
-715.6348239316292
-715.5917259362817
-715.5557771651962
-715.5252213831964
-715.4987135755373
-715.4752102616255
-715.453890957391
-715.4341012920858
-715.415311330756
-715.3970846906969
-715.3790554033674
-715.3609103925497
-715.3423760666795
-715.3232079568883
-715.3031826364662
-715.2820913743028
-715.2597351323375
-715.2359206342088
-715.2104573220315
-715.1831550898534
-715.153822741972
-715.1222671765556
-715.088293342642
-715.0517050629712
-715.0123068560536
-714.9699069263655
-714.9243215169254
-714.875380826455
-714.8229366725741
-714.7668720183616
-714.707112354306
-714.6436387227609
-714.5765018732716
-714.5058366429556
-714.4318751887415
-714.3549572176457
-714.2755349763539
-714.1941706305035
-714.1115239747212
-714.0283293287165
-713.9453620412048
-713.8633970758364

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-721.1503624842621
-704.9203140665784
-701.4838137217082
-700.2009695596803
-699.5118485953975
-699.0595610479277
-698.7327215669367
-698.4847431152808
-698.290536433236
-698.1342977658869
-698.0053065779142
-697.8960420336277
-697.8011243999074
-697.7166448403602
-697.6397207773058
-697.5681950692324
-697.5004298348425
-697.4351634038866
-697.3714098322988
-697.3083874597249
-697.2454675390725
-697.1821369241301
-697.117970737826
-697.0526122265472
-696.9857578667995
-696.9171463757925
-696.8465506810181
-696.7737721856555
-696.6986368661028
-696.6209928807201
-696.5407094720407
-696.4576770191397
-696.3718081495524
-696.2830398550764
-696.1913365747351
-696.09669421119
-695.9991450326293
-695.8987633783624
-695.7956720302923
-695.6900490316074
-695.5821346275704
-695.4722378732075
-695.3607423061281
-695.2481099340478
-695.1348826591338
-695.0216801879586
-694.909193497123
-694.7981730828825
-694.6894115525564
-694.5837206282353
-694.4819033042451
-694.3847226558715
-694.292869516802

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-711.5336708367518
-696.9142857798809
-693.8086709085636
-692.6737337890748
-692.093703154038
-691.7386499372234
-691.5010716090933
-691.3340825202972
-691.2124799157275
-691.1210459120842
-691.0500239902663
-690.9929275922328
-690.9453205358226
-690.9040896587431
-690.8669945496011
-690.8323813565654
-690.7989966624956
-690.7658638189266
-690.732199095245
-690.6973537080089
-690.6607729714464
-690.6219669505174
-690.5804889425779
-690.5359193418902
-690.4878532318921
-690.4358905685988
-690.3796281652425
-690.3186529253827
-690.2525359390485
-690.1808271798787
-690.1030506389183
-690.018699815737
-689.92723357095
-689.828072435655
-689.7205955830519
-689.6041388065114
-689.4779940291934
-689.3414111069611
-689.1936029929751
-689.0337557207282
-688.8610451351706
-688.6746628417194
-688.4738543930254
-688.2579731667946
-688.0265534672941
-687.7794057170088
-687.5167346218034
-687.2392771860481
-686.9484507760975
-686.6464919378482
-686.3365554591591
-686.0227334075996
-685.7099512250435

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [2]:
import math
import numpy
from numpy import random as rand
from sklearn.model_selection import KFold
from functools import reduce
#from model_selection import KFold
from sklearn.metrics import mean_squared_error


class hmm:
    def __init__(self, n_state, obs_symbols, **args):
        """
        Keywords
        :param n_states (int): number of hidden states
        :param output (list, for example["0","1"]): the output symbol notations
        :param mode (string. For example, performance, time, performance+time): the output mode
        :param args: 'Pi' - matrix of initial state probability distribution
                     'T' - matrix of transmission probability
                     'E' - matrix of emission probability
                     'F' - fixed emission probability for the given state {'state1': [0.2, 0.8]}
        :return:
        """

        # Number of hidden states
        self.N = n_state
        # Observation symbols for each type of observation
        # For example, {correct, incorrect} and {fast, slow}
        self.V = obs_symbols
        # Number of observation symbols for each type
        # For example, [2, 5]
        #self.M = map(len, obs_symbols)
        self.M = list(map(len, obs_symbols))
       
        # Number of observation types
        self.n_elements = len(self.V)
        # The mapping of symbols to numbers
        self.symbol_map = []
        # Number of observation types
        for i in range(self.n_elements):
            self.symbol_map.append(dict(zip(self.V[i], range(len(self.V[i])))))
        

        # Initialize transmission probability matrix
        if 'T' in args:
            self.T = args['T']
            if numpy.shape(self.T) != (self.N, self.N):
                raise ValueError("The transmission probability matrix dimension mismatches the given states number.")

            if not numpy.array_equal(self.T.sum(1), numpy.array([1.0] * len(self.T.sum(1)))):
                raise ValueError("The sum of each row in the transmission matrix should equal to 1")
        else:

            raw_T = rand.uniform(0, 1, self.N * self.N).reshape(self.N, self.N)
            raw_T_sum = raw_T.sum(axis=1, keepdims=True)
            self.T = raw_T.astype(float) / raw_T_sum

        self.E = []

        # Initialize emission probability matrix
        if 'E' in args:
            self.E = args['E']            
            if len(self.E) != self.n_elements:
                raise ValueError("There are " + str(self.n_elements) + " in the observations.")

            for i in range(self.n_elements):
                if numpy.shape(self.E[i]) != (self.N, self.M[i]):
                    raise ValueError("The emission probability matrix dimension mismatches the given states number and "
                                     "output symbols number")

                if not numpy.allclose(self.E[i].sum(1), numpy.array([1] * len(self.E[i].sum(1)))):
                    raise ValueError("The sum of each row in the emission probability matrix should equal to 1")
        else:
            for i in range(self.n_elements):
                num = self.M[i]
                print(num)
                raw_E1 = rand.uniform(0, 1, (self.N *num ))
                raw_E = raw_E1.reshape(self.N, self.M[i])
                raw_E_sum = raw_E.sum(axis=1, keepdims=True)
                self.E.append(raw_E.astype(float) / raw_E_sum)
                print("E",self.E )

        # Initialize the initial probability
        if 'Pi' in args:
            self.Pi = args['Pi']

            if len(self.Pi) != self.N:
                raise ValueError("The initial state probability dimension mismatches the given states number.")

            if self.Pi.sum() != 1:
                raise ValueError("The initial state probability does not add up to 1.")

        else:
            raw_Pi = numpy.array([1] * self.N)
            self.Pi = raw_Pi.astype(float) / raw_Pi.sum()

        self._print_HMM("HMM Initialization")

    def _print_HMM(self, label, write_to_file=False):
        """
        Keywords
        :param label (String "The initialized HMM parameters")
        :param write_to_file (boolean): whether to print out to file or not
        :return:
        """
        results = "\n" * 2 + "*" * 24 + "\n" + label + "\n" + "*" * 24 + "\n" \
                  + "\n1) Numerber of hidden states:" + str(self.N) \
                  + "\n2) Number of observable symbols:" + str(self.V) \
                  + "\n3) The symbol mapping in HMM:" + str(self.symbol_map) \
                  + "\n4) The transmission proability matrix T:\n" + str(self.T) \
                  + "\n5) The emission probability matrix E:\n" + str(self.E) \
                  + "\n6) The initial state probability Pi: \n" + str(self.Pi) + "\n"

        print (results)

    def obs_index(self, Obs, Obs_type):
        """
        Convert the observation sequences into sequence using symbols "0", "1" or "2"
        :param Obs:
        :return:
        """
        obs_index_seq = []

        for o in Obs:
            if o not in self.symbol_map[Obs_type]:
                raise ValueError("The observation symbol \"" + o + "\" is not defined in HMM")
            obs_index_seq.append(self.symbol_map[Obs_type][o])

        return obs_index_seq

    def forward(self, Obs, scaling=True, debug=False):
        """
        Calculate the probability of an observation sequence given the model parameters
        P(Obs|hmm)

        Alpha is defined as P(O_1:T,S_T|hmm)

        :param Obs: List. Observation sequence
        :param scaling: boolean. Scale the Alpha matrix to let the column sums to 1
        :param debug: boolean. Whether to print output of each step

        :return:
        """
        if debug:
            print ("\n" * 2 + "*" * 23 + "\n" + "*" * 2 + " FORWARD ALGORITHM " + "*" * 2 + "\n" + "*" * 23 + "\n")

        observation = []
        # The observation sequence using observation symbol notations. It is a list ["1","1","0","1"]

        for i in range(self.n_elements):
            observation.append(self.obs_index(Obs, i))

        T = len(observation[0])
        # create scaling vector
        if scaling:
            c = numpy.zeros([T], float)

        # Initialization
        Alpha = numpy.zeros([self.N, T], float)
        Alpha[:, 0] = self.Pi
        for i in range(self.n_elements):
            Alpha[:, 0] *= self.E[i][:, int(observation[i][0])]

        if scaling:
            c[0] = 1 / Alpha[:, 0].sum()
            Alpha[:, 0] = Alpha[:, 0] * c[0]

        if debug:
            print ("t=0")
            print (Alpha[:, 0])

        # Induction
        for t in range(1, T):
            Alpha[:, t] = numpy.dot(Alpha[:, t - 1], self.T)
            for i in range(self.n_elements):
                Alpha[:, t] *= self.E[i][:, int(observation[i][t])]

            if scaling:
                c[t] = 1 / Alpha[:, t].sum()
                Alpha[:, t] = Alpha[:, t] * c[t]

            if debug:
                print ("t=" + str(t))
                print (Alpha[:, t])

        # Termination
        if scaling:
            log_prob = - reduce((lambda x, y: x + y), numpy.log(c[:T]))

            if debug:
                print ("\nAlpha:")
                print (Alpha)
                print ("\nc:")
                print (c)
                print ("\nP(Obs|hmm)=" + str(log_prob))
                # print "c[T-1]: " + str(c[T-1])
            return (log_prob, Alpha, c)

        else:

            log_prob = numpy.log(numpy.sum(Alpha[:, T - 1]))

            if debug:
                print ("\nAlpha:")
                print (Alpha)
                c = 1.0 / Alpha.sum(0)
                print (c)
                print ("\nP(Obs|hmm)=" + str(log_prob))

            return (log_prob, Alpha)

    def backward(self, Obs, scaling, debug=False, **args):
        """
        Calculate the probability of a partial observation sequence from t+1 to T given the model params.

        Beta is defined as P(O_1:T|S_T, hmm)

        :param Obs: Observation sequence
        :return: Beta
        """
        if debug:
            print ("\n" * 2 + "*" * 24 + "\n" + "*" * 2 + " BACKWARD ALGORITHM " + "*" * 2 + "\n" + "*" * 24 + "\n")

        observation = []
        # The observation sequence using observation symbol notations. It is a list ["1","1","0","1"]
        for i in range(self.n_elements):
            observation.append(self.obs_index([each[i] for each in Obs], i))

        T = len(observation[0])

        if scaling:
            c = numpy.zeros([T], float)
        # Initialization
        Beta = numpy.zeros([self.N, T], float)
        Beta[:, T - 1] = 1
        if scaling:
            c[T - 1] = 1 / Beta[:, T - 1].sum()
            Beta[:, T - 1] = Beta[:, T - 1] * c[T - 1]
        if debug:
            print ("t=" + str(T - 1))
            print (Beta[:, T - 1])

        # Induction

        for t in reversed(range(T - 1)):
            temp = self.T.copy()
            for i in range(self.n_elements):
                temp *= self.E[i][:, int(observation[i][t + 1])]

            Beta[:, t] = numpy.dot(temp, Beta[:, t + 1])
            if scaling:
                c[t] = 1 / Beta[:, t].sum()
                Beta[:, t] = Beta[:, t] * c[t]

            if debug:
                print ("t=" + str(t))
                print (Beta[:, t])

        # if 'c' in args:
        #    Beta = Beta * args['c']

        if scaling:

            if debug:
                print ("\nBeta:")
                print (Beta)

            return Beta
        else:
            if debug:
                print ("\nBeta:")
                print (Beta)
            return Beta

    #Usado en BKT        
    def baum_welch(self, Obs_seq, **args):
        """
        Adjust the model parameters to maximize the probability of the observation sequence given the model

        Define:

        Gamma_t(i) = P(O_1:T, q_t = S_i | hmm) as the probability of in state i at time t and having the
        observation sequence.

        Xi_t(i,j) = P(O_1:T, q_t-1 = S_i, q_t = S_j | hmm) as the probability of transiting from state i
        to state j and having the observation sequence.


        :param Obs_seq: A set of observation sequence
        :param args:
            epochs: number of iterations to perform EM, default is 20
        :return:
        """
        # print "\n"*2+ "*"*24 + "\n" +"*"*1+" Bawn Welch ALGORITHM "+"*"*1 + "\n" + "*"*24 + "\n"
        epochs = args['epochs'] if 'epochs' in args else 100

        updatePi = args['updatePi'] if 'updatePi' in args else True
        updateT = args['updateT'] if 'updateT' in args else True
        updateE = args['updateE'] if 'updateE' in args else True
        debug = args['debug'] if 'debug' in args else False
        epsilon = args['epsilon'] if 'epsilon' in args else 0.001

        LLS = []

        for epoch in range(epochs):
            #print ("Epoch " + str(epoch))

            # Expected number of probability of starting from Si
            exp_si_t0 = numpy.zeros([self.N], float)
            # Expected number of transition from Si
            exp_num_from_Si = numpy.zeros([self.N], float)
            # Expected number of being in Si
            exp_num_in_Si = numpy.zeros([self.N], float)
            # Expected number of transition from Si to Sj
            exp_num_Si_Sj = numpy.zeros([self.N * self.N], float).reshape(self.N, self.N)
            # Expected number of in Si observing symbol Vk
            exp_num_in_Si_Vk = []
            for i in range(self.n_elements):
                exp_num_in_Si_Vk.append(numpy.zeros([self.N, self.M[i]], float))

            LogLikelihood = 0

            for Obs in Obs_seq:
                if debug:
                    print ("\nThe observation sequence is " + str(Obs))
                # log_prob1, Alpha1, c1 = self.forward(Obs, scaling=True, debug=False)

                log_prob, Alpha, c = self.forward(Obs, scaling=True, debug=False)
                LogLikelihood += log_prob
                # print "Log Likelihood is " + str(LogLikelihood)

                Beta = self.backward(Obs, scaling=True, debug=False)

                if debug:
                    print ("\nAlpha:")
                    print (Alpha)
                    print ("\nBeta:")
                    print (Beta)
                    print (len(Beta[0]))

                T = len(Obs)

                observation = []
                # The observation sequence using observation symbol notations. It is a list ["1","1","0","1"]
                for i in range(self.n_elements):
                    observation.append(self.obs_index([each[i] for each in Obs], i))

                #### Calculate Gamma ####
                # Gamma is defined as the probability of
                # being in State Si at time t, given the observation sequence
                # O1, O2, O3, O4 ,....,Ot, and the model parameters

                raw_Gamma = Alpha * Beta
                Gamma = raw_Gamma / raw_Gamma.sum(0)
                if debug:
                    print ("\nGamma")
                    print (Gamma)

                exp_si_t0 += Gamma[:, 0]
                exp_num_from_Si += Gamma[:, :T - 1].sum(1)
                exp_num_in_Si += Gamma.sum(1)

                # The probability in state Si having Observation Oj
                for i in range(self.n_elements):

                    temp = numpy.zeros([self.N, self.M[i]], float)
                    for each in self.symbol_map[i]:#.iterkeys():
                        which = numpy.array([self.symbol_map[i][each] == int(x) for x in observation[i]])
                        temp[:, self.symbol_map[i][each]] = Gamma.T[which, :].sum(0)

                    exp_num_in_Si_Vk[i] += temp

                if debug:
                    print ("\nExpected frequency in state S_i at time 0:\n" + str(exp_si_t0))
                    print ("Expected number of transition from state S_i:\n" + str(exp_num_from_Si))
                    print ("Expected number of time in state S_i:\n" + str(exp_num_in_Si))
                    print ("Expected number of time in state S_i observing V_k:\n" + str(exp_num_in_Si_Vk))

                # Xi is defined as given the model and sequence, the probability of being in state Si at time t,
                # and in state Sj at time t+1

                Xi = numpy.zeros([T - 1, self.N, self.N], float)

                for t in range(T - 1):
                    for i in range(self.N):
                        Xi[t, i, :] = Alpha[i, t] * self.T[i, :]
                        for j in range(self.n_elements):
                            Xi[t, i, :] *= self.E[j][:, int(observation[j][t + 1])]
                        Xi[t, i, :] *= Beta[:, t + 1]
                    Xi[t, :, :] /= Xi[t, :, :].sum()

                for t in range(T - 2):
                    exp_num_Si_Sj += Xi[t, :, :]

                if debug:
                    print ("\nExpected number of transitions from state Si to state Sj: \n" + str(exp_num_Si_Sj))

            # reestimate initial state probabilities
            if updatePi:
                self.Pi = exp_si_t0 / exp_si_t0.sum()
                if debug:
                    print ("\nUpdated Pi:")
                    print (exp_si_t0)

            if updateT:
                T_hat = numpy.zeros([self.N, self.N], float).reshape(self.N, self.N)
                for i in range(self.N):
                    T_hat[i, :] = exp_num_Si_Sj[i, :] / exp_num_from_Si[i]
                    T_hat[i, :] /= T_hat[i, :].sum()
                self.T = T_hat

                if debug:
                    print ("\nUpdated T")
                    print (self.T)

            if updateE:
                for j in range(self.n_elements):
                    E_hat = numpy.zeros([self.N, self.M[j]], float).reshape(self.N, self.M[j])
                    for i in range(self.N):
                        E_hat[i, :] = exp_num_in_Si_Vk[j][i, :] / exp_num_in_Si[i]
                        E_hat[i, :] /= E_hat[i, :].sum()
                    self.E[j] = E_hat
                    if debug:
                        print ("\nUpdated E")
                        print (self.E)
            print (LogLikelihood)
            LLS.append(LogLikelihood)
            if epoch > 1:
                if abs(LLS[epoch] - LLS[epoch - 1]) < epsilon:
                    print ("The loglikelihood improvement falls below threshold, training terminates at epoch " + str(
                        epoch) + "! ")
                    break

        self._print_HMM("After training")
        
        return
    
    #Usado en BKT
    def predict_nlg(self, Obs_seq, debug=False):
        nlg_scores = []
        for Obs in Obs_seq:
            if debug:
                print ("\nThe observation sequence is " + str(Obs))
            if Obs:
                log_prob, Alpha, c = self.forward(Obs, scaling=True, debug=False)
                # print "Posttest score:"
                # print round(Alpha[:, -1][1], 2)
                # nlg = round(Alpha[:, -1][1], 2)
                nlg = Alpha[:, -1][1]
            else:
                nlg = 0
            nlg_scores.append(nlg)
            # print "NLG Score:" + str(round(Alpha[:, -1][1], 2) - hmm.Pi[0])
        return nlg_scores


